In [1]:
import os
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


Loading the API key through Groq. The API is saved in .env file so it is not exposed.

In [2]:
load_dotenv()

True

In [3]:
api_key = os.getenv('GROQ_API_KEY')

In [4]:
chat = ChatGroq(temperature=0, groq_api_key=api_key, model_name="llama3-70b-8192")

Load embedding model

In [5]:
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/paraphrase-MiniLM-L6-v2')

c:\Users\andyu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
comet_ml is installed but `COMET_API_KEY` is not set.


Find path to the persist directory, this should be a directory where you stored your vector database.

In [6]:
persist_directory = '../first attempt rag/chroma_db'

This is just a check. sometimes I had an problem that it could not find the directory.

In [7]:
if not os.path.exists(persist_directory):
    print("Persist directory does not exist.")
else:
    print("Persist directory exists.")

Persist directory exists.


Connecting to already created ChromaDB and setting it up as a retriever(our source of information)

In [8]:
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding_model)

In [9]:
num_documents = vectordb._collection.count()  
print(f"Number of documents in the vector store: {num_documents}")

Number of documents in the vector store: 0


In [10]:
retriever = vectordb.as_retriever()

Attempt of custom prompt template. Just so if the information is not find in the veector database the LLM does not start to give us random answears.

In [11]:
custom_prompt_template = """Use the following pieces of information to answer the user's question. Always answear the question as if you were a human. If you don't know the answer, just say that you don't know, don't try to make up an answer.



Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [12]:
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

prompt = set_custom_prompt()

Setting up the QAchain. What it does it binds everything togeather. Based on prompt we find similarity in the vector database, we get that information and LLM gives us the answear while following the custom prompt.

In [13]:
qa = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={'prompt': prompt}
)

In [14]:
query = "Article whit highest citation count"
result = qa({"query": query})

c:\Users\andyu\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [15]:
print("Answer:", result["result"])

Answer: I don't know.


In [16]:
query = "How many articles were published in 2005"
result = qa({"query": query})
print("Answer:", result["result"])

Answer: I don't know.


Trying out queries to see if we get answears based on our vector database.

In [17]:
query = "Give me articles that talk about software innovation."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: Here are some articles that discuss software innovation:

* "The Future of Software Innovation" by Forbes - This article explores the trends and technologies that are driving software innovation, including artificial intelligence, blockchain, and the Internet of Things.
* "Software Innovation: The Key to Digital Transformation" by Harvard Business Review - This article argues that software innovation is essential for businesses to stay competitive in today's digital landscape.
* "The Top 10 Software Innovations of the Past Decade" by Wired - This article highlights some of the most significant software innovations of the past decade, including cloud computing, mobile apps, and virtual reality.
* "How to Foster a Culture of Software Innovation" by McKinsey - This article provides guidance on how businesses can create a culture that encourages software innovation, including strategies for talent development and innovation pipelines.
* "The Role of AI in Software Innovation" by Te

In [18]:
query = "What is article with the name Data Science for Scoial Good about."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: I don't know.


In [19]:
import textwrap

In [20]:
query = "What is the article with the name Data Science for Social Good about."
result = qa({"query": query})
wrapped_answer = textwrap.fill(result["result"], width=80)
print("Answer:\n", wrapped_answer)

Answer:
 I don't know.


In [21]:
query = "Give me full abstract of article Data Science for Social Good."
result = qa({"query": query})
wrapped_answer = textwrap.fill(result["result"], width=80)
print("Answer:\n", wrapped_answer)

Answer:
 I apologize, but I don't have the abstract of the article "Data Science for
Social Good" to provide. Can you please provide more context or information
about the article, such as the author or publication date, so I can try to find
it for you?


In [22]:
query = "Give me an name of an article that talks about  research that considers the interplay between relevant data science research genres, social good challenges, and different levels of sociotechnical abstraction, and highlighting the lack of research focuon social good challenges in the field of data science."
result = qa({"query": query})
wrapped_answer = textwrap.fill(result["result"], width=80)
print("Answer:\n", wrapped_answer)

Answer:
 I'm not aware of a specific article that exactly matches the description you
provided. If you're interested in learning more about the intersection of data
science and social good, I'd be happy to help you search for relevant resources
or provide more general information on the topic.


In [23]:
query = "Give me full abstract of article Data Science for Social Good."
result = qa({"query": query})
wrapped_answer = textwrap.fill(result["result"], width=80)
print("Answer:\n", wrapped_answer)

Answer:
 I apologize, but I don't have the abstract of the article "Data Science for
Social Good" to provide. Can you please provide more context or information
about the article, such as the author or publication date, so I can try to find
it for you?


In [24]:
query = "How similar are the these two articles:Essence: facilitating software innovation and Data Science for Social Good."
result = qa({"query": query})
wrapped_answer = textwrap.fill(result["result"], width=80)
print("Answer:\n", wrapped_answer)

Answer:
 After reading both articles, I can say that they are quite different in terms of
their focus and content. "Essence: facilitating software innovation" appears to
be a technical article discussing a specific software development methodology,
whereas "Data Science for Social Good" seems to be a more general article about
the application of data science in social impact projects. The topics, tone, and
language used in both articles are distinct, so I wouldn't say they are similar
at all.


In [25]:
query = "Give me 3 articles that talk about the similar topic as this one and also tell me why is it similar Essence: facilitating software innovation"
result = qa({"query": query})
wrapped_answer = textwrap.fill(result["result"], width=80)
print("Answer:\n", wrapped_answer)

Answer:
 Here are three articles that discuss similar topics related to facilitating
software innovation:  1. "Accelerating Digital Innovation: A Framework for
Software Development" by McKinsey - This article is similar because it explores
ways to speed up software development and innovation, which aligns with the
essence of facilitating software innovation.  2. "The Future of Software
Development: Trends and Opportunities" by Forbes - This article is similar
because it discusses the latest trends and opportunities in software
development, which is closely related to facilitating software innovation.  3.
"How to Create a Culture of Innovation in Software Development" by Harvard
Business Review - This article is similar because it focuses on creating a
culture that encourages innovation in software development, which is a key
aspect of facilitating software innovation.  These articles are all similar
because they focus on improving software development and innovation, which is
the core 